#  Text Classification

In [1]:
#import
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from xgboost.sklearn import XGBClassifier
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline

In [2]:
class_data_df = pd.read_csv("all_uni_df.csv")[['research_topic','department',]]
class_data_df.head(3)

,research_topic,department
0,"'sociology', 'pedagogics', 'university', 'soci...",Department of Education
1,"'pedagogics', 'Teaching', 'candidacy', 'educat...",Department of Social & Policy Sciences
2,"'orange juice', 'phytosterols', 'fouling', 'ul...",Department of Chemical Engineering


In [3]:
#Medical
class_data_df = class_data_df.replace(to_replace =['School of Human and Health Sciences','Department of Pharmacy & Pharmacology','Department of Pharmacy','Department of Nursing and Midwifery','Department of Allied Health Professions, Sport and Exercise','Department for Health','Department of Biology & Biochemistry','Department of Biological and Geographical Sciences'],  value ="Medical")

#Finance
class_data_df = class_data_df.replace(to_replace =['School of Economics, Finance and Accounting','Faculty Research Centre for Financial & Corporate Integrity','Department of Economics','Department of Accounting, Finance and Economics'],  value ="Finance")

#Science and Technology
class_data_df = class_data_df.replace(to_replace =['School of Computing, Electronics and Maths','School of Applied Sciences','Department of Physics','Department of Optometry and Vision Sciences','Department of Mathematical Sciences','Department of Computer Science','Department of Chemistry','Department of Chemical Sciences'],  value ="Science and Technology")

#Arts
class_data_df = class_data_df.replace(to_replace =['Faculty Research Centre in Postdigital Cultures','School of Art, Design and Architecture','Faculty of Science','Faculty of Humanities and Social Sciences','Department of Psychology','Department of History, English, Linguistics and Music','Department of Art and Communication'],  value ="Arts")

#Engineering
class_data_df = class_data_df.replace(to_replace =['School of Mechanical, Aerospace and Automotive Engineering','School of Computing and Engineering','Department of Mechanical Engineering','Department of Engineering and Technology','Department of Electronic & Electrical Engineering','Department of Chemical Engineering','Department of Architecture & Civil Engineering','Department of Architecture and 3D Design'],  value ="Engineering")

#Management
class_data_df = class_data_df.replace(to_replace =['School of Marketing and Management','Management','Department of Media and Performance','Department of Management','Department of Logistics, Marketing, Hospitality and Analytics'],  value ="Management")

#Education
class_data_df = class_data_df.replace(to_replace =['School of Education and Professional Development','Department of Initial Teacher Education','Department of Education and Community Studies','Department of Education','Centre for Global Learning, Education and Attainment'],  value ="Education")

#Drop
drop = ['k.christmann@hud.ac.uk','-','Department of Social & Policy Sciences','Department of Fashion and Textiles','Department of Behavioural and Social Sciences','Designability','Faculty Research Centre for Built & Natural Environment','Faculty Research Centre in Fluid and Complex Systems','Huddersfield Business School','Institute for Future Transport and Cities','Institute for Policy Research (IPR)','Non Academic Depts','Politics, Languages & International Studies','School of Energy, Construction and Environment','School of Music, Humanities and Media','School of Strategy and Leadership','The Law School','Vice-Chancellor\'s Office','k.christmann@hud.ac']
for d in drop:
    class_data_df = class_data_df[class_data_df.department != d]

class_data_df

,research_topic,department
0,"'sociology', 'pedagogics', 'university', 'soci...",Education
2,"'orange juice', 'phytosterols', 'fouling', 'ul...",Engineering
3,"'Feedstocks', 'Biomass', 'Fermentation', 'Econ...",Engineering
4,"'Terminator Codon', 'Codon', 'Permethrin', 'Ba...",Medical
5,"'Neurons', 'Specifications', 'Silicon', 'Netwo...",Science and Technology
...,...,...
995,"'China', 'Uncertainty', 'Volatility spillover'...",Finance
996,"'employability', 'education', 'experience', 'l...",Education
997,"'Lactams', 'Hydrolysis', 'Acids', 'Polysacchar...",Science and Technology
998,"'Out-of-Hospital Cardiac Arrest', 'Ambulances'...",Medical


# -------------------------------------------------------

In [4]:
data_improved = class_data_df.reset_index(drop=True)
data_improved

,research_topic,department
0,"'sociology', 'pedagogics', 'university', 'soci...",Education
1,"'orange juice', 'phytosterols', 'fouling', 'ul...",Engineering
2,"'Feedstocks', 'Biomass', 'Fermentation', 'Econ...",Engineering
3,"'Terminator Codon', 'Codon', 'Permethrin', 'Ba...",Medical
4,"'Neurons', 'Specifications', 'Silicon', 'Netwo...",Science and Technology
...,...,...
744,"'China', 'Uncertainty', 'Volatility spillover'...",Finance
745,"'employability', 'education', 'experience', 'l...",Education
746,"'Lactams', 'Hydrolysis', 'Acids', 'Polysacchar...",Science and Technology
747,"'Out-of-Hospital Cardiac Arrest', 'Ambulances'...",Medical


In [5]:
data_improved.research_topic = data_improved.research_topic.replace('[0-9]', '', regex=True)

In [6]:
lenght = []

for i in range(len(data_improved)):
    lenght.append(len(data_improved.research_topic[i].split()))

In [7]:
data_improved['lenght'] = lenght

In [8]:
data_improved.head(5)

,research_topic,department,lenght
0,"'sociology', 'pedagogics', 'university', 'soci...",Education,67
1,"'orange juice', 'phytosterols', 'fouling', 'ul...",Engineering,29
2,"'Feedstocks', 'Biomass', 'Fermentation', 'Econ...",Engineering,110
3,"'Terminator Codon', 'Codon', 'Permethrin', 'Ba...",Medical,60
4,"'Neurons', 'Specifications', 'Silicon', 'Netwo...",Science and Technology,13


In [9]:
data_improved = data_improved[data_improved.lenght>10]
len(data_improved)

669

In [10]:
departments = class_data_df.groupby('department')
print("There's {} departments." .format(departments.ngroups))
print(departments.size())

There's 9 departments.
department
Arts                                            112
Centre for Agroecology, Water and Resilience     20
Centre for Trust, Peace and Social Relations     10
Education                                        43
Engineering                                     133
Finance                                          85
Management                                       82
Medical                                         145
Science and Technology                          119
dtype: int64


In [385]:
X, y = class_data_df.research_topic, class_data_df.department;

#tokenize and create bag of words
vectorizer = CountVectorizer(stop_words="english")

X = vectorizer.fit_transform(X)

In [386]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [387]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

                                              precision    recall  f1-score   support

                                        Arts       0.93      0.63      0.75        43
Centre for Agroecology, Water and Resilience       1.00      1.00      1.00         4
Centre for Trust, Peace and Social Relations       1.00      1.00      1.00         2
                                   Education       0.61      0.73      0.67        15
                                 Engineering       0.73      0.75      0.74        48
                                     Finance       0.96      0.86      0.91        28
                                  Management       0.64      0.76      0.70        21
                                     Medical       0.61      0.77      0.68        30
                      Science and Technology       0.63      0.65      0.64        34

                                    accuracy                           0.73       225
                                   macro avg       0

In [388]:
dump(vectorizer, "preprocessor.joblib")
dump(model, "uni_classifier.joblib")

['uni_classifier.joblib']

In [389]:
preprocessor = load("preprocessor.joblib")
model = load("uni_classifier.joblib")

def predict_topic():
  text = input("Enter text to be predicted: ")
  text = preprocessor.transform([text])
  prediction = model.predict(text)[0]
  print("This text is most likely about {}." .format(prediction))

In [ ]:
predict_topic()